In [1]:
import tensorflow as tf
tf.random.set_seed(230)

### Import data - Load MNIST Train and Test datasets 

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [15]:
print("Training data type: %s;\nTraining data shape: %s" % (x_train.dtype, x_train.shape))
print("\nTest data type: %s;\nTest data shape: %s" % (x_test.dtype, x_test.shape))

Training data type: uint8;
Training data shape: (60000, 28, 28)

Test data type: uint8;
Test data shape: (10000, 28, 28)


### Preprocess data

In [3]:
def preprocess_features(x, y):
    x = tf.cast(x, tf.float32)/255.0
    y = tf.cast(y, tf.int64)
    return x, y

def preprocess(x, y, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(preprocess_features)
    if mode == "train":
        ds = ds.shuffle(10000)
    ds = ds.batch(256)
    return ds

In [4]:
train_ds = preprocess(x_train, y_train)
val_ds = preprocess(x_test, y_test, "test")

### Define Model

#### Option 1 - Default Sequential Model from Keras

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10)])

#### Option 2 - Custom Model inherited from Keras Model Class  

In [5]:
class Model(tf.keras.Model):

    def __init__(self, in_shape, layers, out_shape):
        super().__init__()
        
        self.model_layers = []
        self.model_layers.append(tf.keras.layers.Reshape(target_shape=(in_shape,)))
        for layer in layers:
            units = layer[0]
            activation = layer[1]
            self.model_layers.append(tf.keras.layers.Dense(units=units, activation=activation))
        self.model_layers.append(tf.keras.layers.Dense(units=out_shape, activation=None))

    def call(self, inputs):
        outputs = inputs
        for layer in self.model_layers:
            outputs = layer(inputs)
            inputs = outputs
        return outputs

In [6]:
in_shape = 28*28
ls = [[100, "relu"], [100, "relu"]]
out_shape = 10
model = Model(in_shape, ls, out_shape)

### Define Loss and Optimizer

In [6]:
optimizer = tf.keras.optimizers.Adam(0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

### Training

#### Option 1 - Default Keras API using .compile and .fit

In [7]:
metrics = ["accuracy"]
epochs = 5

In [11]:
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Train for 235 steps, validate for 40 steps
Epoch 1/5
235/235 [==============================] - 9s 40ms/step - loss: 0.4343 - accuracy: 0.8827 - val_loss: 0.1995 - val_accuracy: 0.9398
Epoch 2/5
235/235 [==============================] - 9s 37ms/step - loss: 0.1707 - accuracy: 0.9512 - val_loss: 0.1354 - val_accuracy: 0.9579
Epoch 3/5
235/235 [==============================] - 9s 39ms/step - loss: 0.1227 - accuracy: 0.9635 - val_loss: 0.1094 - val_accuracy: 0.9647
Epoch 4/5
235/235 [==============================] - 9s 36ms/step - loss: 0.0952 - accuracy: 0.9715 - val_loss: 0.0968 - val_accuracy: 0.9689
Epoch 5/5
235/235 [==============================] - 8s 36ms/step - loss: 0.0782 - accuracy: 0.9763 - val_loss: 0.0913 - val_accuracy: 0.9703


#### Option 2 - Custom Training using Gradient Tape

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

In [8]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_fn(y, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y, predictions)

In [9]:
@tf.function
def val_step(x, y):
    predictions = model(x)
    loss = loss_fn(y, predictions)

    val_loss(loss)
    val_accuracy(y, predictions)

In [10]:
epochs = 5

for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()

    for x, y in train_ds:
        train_step(x, y)

    for val_x, val_y in val_ds:
        val_step(val_x, val_y)

    print('Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        val_loss.result(),
                        val_accuracy.result() * 100))

Epoch 1, Loss: 0.43431714177131653, Accuracy: 88.27000427246094, Val Loss: 0.19948408007621765, Val Accuracy: 93.9800033569336
Epoch 2, Loss: 0.1707475483417511, Accuracy: 95.11666870117188, Val Loss: 0.13541637361049652, Val Accuracy: 95.79000091552734
Epoch 3, Loss: 0.12271910905838013, Accuracy: 96.35499572753906, Val Loss: 0.10936269909143448, Val Accuracy: 96.47000122070312
Epoch 4, Loss: 0.09524733573198318, Accuracy: 97.1483383178711, Val Loss: 0.09678876399993896, Val Accuracy: 96.88999938964844
Epoch 5, Loss: 0.07820867747068405, Accuracy: 97.63333129882812, Val Loss: 0.09128781408071518, Val Accuracy: 97.02999877929688
